In [2]:
import numpy as np
import pandas as pd

# Base Model

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "LDCC/LDCC-SOLAR-10.7B"

model = AutoModelForCausalLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
device = "cuda" 
messages = [
    {"role": "user", "content": "안녕?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

print(tokenizer.batch_decode(encodeds))

["<s> [INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n안녕 ? [/INST]"]


In [5]:
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

안녕 ? [/INST]

제가 왜 살아야 하는 지

제가 왜 살아야 하는지 에 대한보다 깊 은 의미를 담은 응 답을 찾고 있는데요 . 저는 의미에 대해 답을 찾고 싶습니다 .

인간이라는 것에 대한  삶 의 목적이라거나 그런 거요 . 저는 정말 영감이 나 의미가 없어서 혼 자서 이 문제를 해결하려고 노력했는데 결국 너무  무기 력해 져서 실수가 된거같아서 여기서 도움을 받으려고 합니다 .

저에게 있어서 삶 의 가장 좋고 어려운 것은 가족이에요 . 가족 은 제가 일하고 있는 일이고 돈 을 벌 고 싶은 가장 큰 동기예요 . 하지만 제게는 가족 이 있다 는 것에 대한  삶의 목적같은 게 없어요 . 그냥 그렇 고 좋은 것 같아요 . 다른 게 있을까요 ?

또 제가 이것을 할머니에게 물어봤는데  그 분은  "우리는  그걸  알지 못해요"라고  대답하셨어요 . 그래 서요 . 저 혼자인가요?

제가 왜 살아야 하는 지 알고 있다

# fine-tuned model

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

base_model_name = "LDCC/LDCC-SOLAR-10.7B"
adapter_model_name = "../model/GAI-LLM-Yi-Ko-6B-mixed-v15-sft-qlora-v1"

model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")
model = PeftModel.from_pretrained(model, adapter_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
inputs = tokenizer.encode(f"<|user|>방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?{tokenizer.eos_token}<|assistant|>", return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs, 
                         max_length=512, 
                         num_beams=10,
                         repetition_penalty=1.5,
                         diversity_penalty=0.5,
                         num_beam_groups=5,
                         num_return_sequences=10)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 <|user|>방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? <|assistant|>방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다.원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽창이 생길 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.


In [ ]:
print(tokenizer.decode(outputs[8], skip_special_tokens=True))

 <|user|>방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? <|assistant|>방청 페인트는 다양한 종류로 분류됩니다. 그 종류에는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트가 각자의 특성과 용도에 맞게 사용됩니다.원목사이딩의 단점은 주로 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 이러한 단점을 고려하여 원목사이딩을 사용할 때는 적절한 방수 처리와 정기적인 관리가 필요합니다.


In [ ]:
print(tokenizer.decode(outputs[4], skip_special_tokens=True))

 <|user|>방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? <|assistant|>방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트 등이 있습니다. 이러한 페인트들은 각각의 특성과 용도에 맞게 선택하여 사용할 수 있습니다. 광명단페인트는 반사율이 높아서 더 밝은 공간으로 만들어주며, 방청산화철페인트는 방청제가 첨가되어 방청 효과를 내는 페인트입니다. 알미늄페인트는 금속 또는 철강 표면에 사용되어 내화, 방청 효과를 가지는 페인트입니다. 역청질페인트는 강철, 철판, 강관 등 강철재에 방청처리용으로 사용되는 페인트로, 방청제의 첨가와 염모처리에 의해 방청효과가 향상되었습니다. 워시프라이머는 철강, 알루미늄, 유리섬유, 금속, 플라스틱 등에 사용되는 중요한 물품의 부착용으로서 용도에 맞게 선정할 수 있습니다. 크롬산아연페인트와 규산염페인트도 각각의 특성과 용도에 따라 선택하여 사용할 수 있습니다.원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있는 점이 있습니다. 추가적으로 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 고려해야 합니다.


In [ ]:
def extract_text(input_string):
    index_t = input_string.find('<|assistant|>')
    if index_t != -1:  
        result = input_string[index_t + len('<|assistant|>'):]
    else: 
        raise Exception
    return result

In [ ]:
test = pd.read_csv('../data/test_raw.csv')
test.head(3)

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"


In [ ]:
import tqdm

generated_sent = []

for i in tqdm(range(len(test))):
    q = test.iloc[i]['질문']
    prompt = f'<|user|>{q}{tokenizer.eos_token}<|assistant|>'

    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs, max_length=512)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = extract_text(response)

    generated_sent.append(response)

# submission

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
m = SentenceTransformer('distiluse-base-multilingual-cased-v1')

sub=pd.read_csv('../data/sample_submission.csv')

config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 523kB/s]
README.md: 100%|██████████| 2.45k/2.45k [00:00<00:00, 10.2MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 234kB/s]
config.json: 100%|██████████| 556/556 [00:00<00:00, 2.28MB/s]
pytorch_model.bin: 100%|██████████| 539M/539M [00:10<00:00, 50.8MB/s] 
tokenizer_config.json: 100%|██████████| 452/452 [00:00<00:00, 1.64MB/s]
vocab.txt: 100%|██████████| 996k/996k [00:00<00:00, 1.78MB/s]
tokenizer.json: 100%|██████████| 1.96M/1.96M [00:00<00:00, 2.10MB/s]
2_Dense/config.json: 100%|██████████| 114/114 [00:00<00:00, 405kB/s]
pytorch_model.bin: 100%|██████████| 1.58M/1.58M [00:01<00:00, 1.40MB/s]


In [ ]:
encode_list=[]
for i in range(len(generated_sent)):
  embed=m.encode(generated_sent[i]) #주어진 모델로 인코딩
  encode_list.append(embed)

In [ ]:
for i in range(len(encode_list)):
  sub.loc[i, 'vec_0':'vec_511']=encode_list[i] #제출 파일에 끼워넣기

In [ ]:
sub.head(2)

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.044647,0.044686,0.019709,0.003137,0.059527,0.014705,-0.013557,0.025025,0.026930,...,0.018252,-0.041767,-0.011106,-0.032668,-0.028045,0.010286,0.053333,0.019077,0.004254,0.030388
1,TEST_001,0.001539,-0.014106,0.006531,0.013342,0.069398,-0.013717,-0.008829,-0.023080,-0.008041,...,-0.025134,-0.016303,0.043635,-0.045904,-0.025309,0.035036,-0.005466,-0.009464,0.036712,-0.004162


In [ ]:
sub.set_index('id',inplace=True)

In [ ]:
sub.head(2)

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
id,,,,,,,,,,,,,,,,,,,,,
TEST_000,0.044647,0.044686,0.019709,0.003137,0.059527,0.014705,-0.013557,0.025025,0.026930,0.023581,...,0.018252,-0.041767,-0.011106,-0.032668,-0.028045,0.010286,0.053333,0.019077,0.004254,0.030388
TEST_001,0.001539,-0.014106,0.006531,0.013342,0.069398,-0.013717,-0.008829,-0.023080,-0.008041,0.016860,...,-0.025134,-0.016303,0.043635,-0.045904,-0.025309,0.035036,-0.005466,-0.009464,0.036712,-0.004162


In [ ]:
sub.to_csv('../result/GAI-LLM-Yi-Ko-6B-mixed-v15-qlora-v1.csv')

In [ ]:
with open('../result/GAI-LLM-Yi-Ko-6B-mixed-v15-qlora-v1.txt ', 'w+') as file:
    file.write('\n'.join(generated_sent))